In [1]:
from pdf2image import convert_from_path
import pandas as pd
import fitz, re
from PIL import Image
import easyocr
from datetime import datetime

In [2]:
def scrapPdf(nombre):
  
    # leer pdf
    doc = fitz.open(nombre)
    page = doc.loadPage(0) 
    text = page.getText('text')
    # extración  información página 0
    a1 = 'La desagregación'
    b1 = 'INFOGRAFÍA'
    ca1 = text.index(a1) + len(a1)
    cb1 = text.index(b1)
    infoParcial = text[cb1:ca1].split('\n')
    auxInfo =infoParcial[1].split('-')
    info = [infoParcial[0].split(' ')[1][2:],auxInfo[1][6:17].strip(),auxInfo[0][6:17].strip()]
    numInfo = info[0]
    fecha = info[1]
    namef = datetime.strptime(fecha, '%d/%m/%Y').date().strftime('%A')

    cadena_A = 'Casos confirmados por grupo etario'
    cadena_B = 'SITUACIÓN NACIONAL POR  COVID-19' 
    c_A = text.index(cadena_A)
    c_B = text.rindex(cadena_B)
    datos_provincias = text[c_A:c_B]
    datos_provincias= datos_provincias[len(cadena_A):]
    if 'Azuay' in datos_provincias:
        datos_provincias = datos_provincias[datos_provincias.index('Azuay'):datos_provincias.index('Total')]

        f = lambda A, n=4: [A[i:i+n] for i in range(0, len(A), n)]
        salidatexto = datos_provincias.split('\n')
        salida2 = f(salidatexto)
        salida2 = salida2[:-1]

        p = list(map(lambda x: x[0],salida2 ) )
        c = list(map(lambda x: x[1].replace('.','') ,salida2 ) )
        f = list(map(lambda x: x[2].replace('.',''),salida2 ) )
        fp = list(map(lambda x: x[3].replace('.',''),salida2 ))

        out2Dict = {'Nro.Infografía':numInfo,'fecha':fecha,'Provincia':p,'Confirmados':c,'Fallecidos':f,'FallecidosProbables':fp,'NombreDia':namef}
        return  out2Dict
    else:
        poppler_path='C:/Program Files/poppler-0.68.0/bin'
        pages = convert_from_path(nombre, dpi =500,poppler_path=poppler_path,first_page =0 ,last_page=1)
        pages[0].save('out.jpg', 'JPEG')
        img = Image.open('out.jpg')
        outcut = img.crop((1585,1220,3290,2920))
        outcut.save('outcut.jpg', 'JPEG')
        reader = easyocr.Reader(['es'])
        dd= reader.readtext('outcut.jpg')
        
        texto_aux = ''
        for r in dd:
            texto_aux += r[1] + ' '
        texto_aux =texto_aux.replace('.','')
#         end_line = texto_aux[texto_aux.index('Total'):].strip().rsplit(' ',3)# el-> end line
        list_pro = re.findall('[A-Z]\D+ \d+ \d+ \d+',texto_aux)
        list_list_pro = list(map(lambda x: x.strip().rsplit(' ',3),list_pro))
        for y in list_list_pro:
            if len(y)==2 :
                y.append('0')
            if len(y)==3 :
                y.append('0')
#         list_list_pro.append(end_line)
        p2 = list(map(lambda z: z[0],list_list_pro ) )
        c2 = list(map(lambda z: z[1] ,list_list_pro ) )
        f2 = list(map(lambda z: z[2].replace(',','') , list_list_pro ) )
        fp2 = list(map(lambda z: z[3].replace(',',''), list_list_pro ))
        
        out2Dict2 = {'Nro.Infografía':numInfo,'fecha':fecha,'Provincia':p2,'Confirmados':c2,'Fallecidos':f2,'FallecidosProbables':fp2,'NombreDia':namef}

        return  out2Dict2
        
    

In [3]:
salida = scrapPdf('INFOGRAFIA-NACIONALCOVID19-COE-NACIONAL-08h00-07022021.pdf')
salida

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


{'Nro.Infografía': '346',
 'fecha': '07/02/2021',
 'Provincia': ['Azuay',
  'Bolívar',
  'Cañar',
  'Carchi',
  'Chimborazo',
  'Cotopaxi',
  'El Oro',
  'Esmeraldas',
  'Galápagos',
  'Imbabura',
  'Loja',
  'Morona Santiago',
  'Napo',
  'Orellana',
  'Pastaza',
  'Pichincha',
  'Santa Elena',
  'Sto Domingo Tsác',
  'Sucumbios',
  'Tungurahua'],
 'Confirmados': ['16335',
  '3184',
  '3631',
  '5252',
  '4438',
  '7428',
  '10979',
  '5579',
  '1270',
  '7898',
  '9287',
  '4235',
  '1975',
  '2220',
  '2540',
  '90325',
  '3080',
  '7036',
  '3278',
  '9470'],
 'Fallecidos': ['280',
  '75',
  '109',
  '171',
  '387',
  '353',
  '647',
  '247',
  '8',
  '236',
  '273',
  '41',
  '80',
  '57',
  '67',
  '2122',
  '408',
  '419',
  '108',
  '404'],
 'FallecidosProbables': ['23',
  '15',
  '6',
  '3',
  '126',
  '80',
  '213',
  '56',
  '3',
  '17',
  '48',
  '0',
  '3',
  '20',
  '17',
  '312',
  '280',
  '119',
  '3',
  '326'],
 'NombreDia': 'Sunday'}

In [4]:
pd.DataFrame(salida)

,Nro.Infografía,fecha,Provincia,Confirmados,Fallecidos,FallecidosProbables,NombreDia
0,346,07/02/2021,Azuay,16335,280,23,Sunday
1,346,07/02/2021,Bolívar,3184,75,15,Sunday
2,346,07/02/2021,Cañar,3631,109,6,Sunday
3,346,07/02/2021,Carchi,5252,171,3,Sunday
4,346,07/02/2021,Chimborazo,4438,387,126,Sunday
5,346,07/02/2021,Cotopaxi,7428,353,80,Sunday
6,346,07/02/2021,El Oro,10979,647,213,Sunday
7,346,07/02/2021,Esmeraldas,5579,247,56,Sunday
8,346,07/02/2021,Galápagos,1270,8,3,Sunday
9,346,07/02/2021,Imbabura,7898,236,17,Sunday


In [24]:
'Ecuador' in salida
# if salida.index('Azuay') == False:
#     print('no existe')

True

In [53]:
list_list_pro


[['Azuay', '15658', '261', '22'],
 ['Bolívar', '2905', '74', '15'],
 ['Cañar', '3421', '109', '6'],
 ['Carchi', '4928', '163', '3'],
 ['Chimborazo', '4143', '367', '125'],
 ['Cotopaxi', '6972', '338', '79'],
 ['Oro', '10162', '625', '213'],
 ['Esmeraldas', '5382', '246', '56'],
 ['Galápagos', '1239', '8', '3'],
 ['Guayas', '31321', '1943', '1702'],
 ['Imbabura', '7367', '229', '17'],
 ['Loja', '8886', '258', '48'],
 ['Ríos', '5876', '365', '246'],
 ['Manabí', '17261', '1402', '1071'],
 ['Santiago', '4159', '39', '0'],
 ['Napo', '1913', '80', '3'],
 ['Orellana', '2181', '55', '19'],
 ['Pastaza', '2511', '67', '17'],
 ['Pichincha', '86049', '2096', '312'],
 ['Elena', '3016', '407', '280'],
 ['Tsác', '6704', '404', '118'],
 ['Sucumbíos', '3230', '108', '3'],
 ['Tungurahua', '8941', '387', '320'],
 ['Chinchipe', '1775', '56', '1'],
 ['Total general', '246000', '10087', '4679']]

In [50]:
list_list_pro
p = list(map(lambda x: x[0],list_list_pro ) )
c = list(map(lambda x: x[1],list_list_pro ) )
f = list(map(lambda x: x[2],list_list_pro ) )
fp = list(map(lambda x: x[3],list_list_pro ))

In [54]:
p

['Azuay',
 'Bolívar',
 'Cañar',
 'Carchi',
 'Chimborazo',
 'Cotopaxi',
 'Oro',
 'Esmeraldas',
 'Galápagos',
 'Guayas',
 'Imbabura',
 'Loja',
 'Ríos',
 'Manabí',
 'Santiago',
 'Napo',
 'Orellana',
 'Pastaza',
 'Pichincha',
 'Elena',
 'Tsác',
 'Sucumbíos',
 'Tungurahua',
 'Chinchipe',
 'Total general']

In [51]:
out2Dict = {'Provincia':p,'Confirmados':c,'Fallecidos':f,'FallecidosProbables':fp}

In [52]:
out2Dict

{'Provincia': ['Azuay',
  'Bolívar',
  'Cañar',
  'Carchi',
  'Chimborazo',
  'Cotopaxi',
  'Oro',
  'Esmeraldas',
  'Galápagos',
  'Guayas',
  'Imbabura',
  'Loja',
  'Ríos',
  'Manabí',
  'Santiago',
  'Napo',
  'Orellana',
  'Pastaza',
  'Pichincha',
  'Elena',
  'Tsác',
  'Sucumbíos',
  'Tungurahua',
  'Chinchipe',
  'Total general'],
 'Confirmados': ['15658',
  '2905',
  '3421',
  '4928',
  '4143',
  '6972',
  '10162',
  '5382',
  '1239',
  '31321',
  '7367',
  '8886',
  '5876',
  '17261',
  '4159',
  '1913',
  '2181',
  '2511',
  '86049',
  '3016',
  '6704',
  '3230',
  '8941',
  '1775',
  '246000'],
 'Fallecidos': ['261',
  '74',
  '109',
  '163',
  '367',
  '338',
  '625',
  '246',
  '8',
  '1943',
  '229',
  '258',
  '365',
  '1402',
  '39',
  '80',
  '55',
  '67',
  '2096',
  '407',
  '404',
  '108',
  '387',
  '56',
  '10087'],
 'FallecidosProbables': ['22',
  '15',
  '6',
  '3',
  '125',
  '79',
  '213',
  '56',
  '3',
  '1702',
  '17',
  '48',
  '246',
  '1071',
  '0',
  '